Q-learning agent solving the FrozenLake8x8-v0 environment from the Gym package. 
Main classes: `discreteQlearningAgent` and `gymTrainer`. 

To do:
- Inherit all agents from a common (abstract) class. Check if a loaded agent is correct before the training (or test).
- Add a function that checks sizes of state and action spaces in a given (discrete) environments and test if the same agent implementations can be used in other environments.
- Implement other agents:
    - SARSA.
    - Policy gradient.
    - Model based (e.g., Dyna-Q).
    - A better handcrafted agent (graph search + heuristics).  
- Add and test a potential-based shaping reward function [Ng, A. Y., Harada, D., & Russell, S. (1999, June). Policy invariance under reward transformations: Theory and application to reward shaping. In Icml (Vol. 99, pp. 278-287).](https://people.eecs.berkeley.edu/~pabbeel/cs287-fa09/readings/NgHaradaRussell-shaping-ICML1999.pdf)
- Non-tabular (functional approximator) training for randomized maps.
- Functional approximators for non-discrete environments.

Comments
- Looking at the Q function I can see some interesting (rather simple) tactics. For example, it's often better not to go directly "where we want", but instead to go in the direction opposite to H. 
- Two related reasons why it is useful to switch between greedy and stochastic policies:
    - We test how the greedy (with respect to the estimated Q-function) performs. That way we can gauge the real progress.
    - Stochastic (softmax) policy rarely gets to the goal, because it is not easy to tune $\beta$ (this is the reason why I randomize $\beta$). This slows down the traning. Greedy policies, on the other hand, may not explore enough. This is why it may be a good idea to interleave both policies during training.

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import pickle

# By default the FrozenLake environment has an internal maximum of 200 steps (done is set to True after 200 steps).
# Because of that, we do not really need the constant below. However, it is possible to remove this restriction in the environment. 
# In that case we can restrict the number of steps taken in the environment at the level of this module using the constant below.
MAX_LEN_EPISODE = 201    

# In FrozenLake environment actions are encoded as follows:
# 0: LEFT
# 1: DOWN
# 2: RIGHT
# 3: UP

# ------------RL agent (tabular Q-learning)-----------------
class discreteQlearningAgent:
    """A class representing an agent solving the task defined in the OpenAI Gym FrozenLake8x8-v0 environment. 

    Attributes
    ----------
    ... TO DO !!!!!!

    Methods
    -------
    new_episode(state=0):
        Resets attributes representing state (s) and action (a). Should be 
        called at the beginning of each episode.
    set_alpha(alpha=0, step = 0, scheduler=True):
        Sets a new value of the learning rate (alpha).
    set_beta(beta=0, step = 0, scheduler=True):
        Sets a new value of the inverse temperature (beta).
    select_action(greedy=False):
        Selects action.
    update(self, obs, r, done, learn = True):
        Update the state and Q-function (observation + learning).

    Comments
    --------
    Currently, only tabular Q-learning is implemented. This is fine if the map is fixed, but 
    will not generalize to new maps. A more powerful representation with a functional 
    approximation (ConvNet?) is needed to solve a more general (and much more interesting!) 
    problem of an environment in which the map is randomly generated in each episode.

    This implementation is not very general. In particular, the number of states and actions 
    is not inferred from the environment and as such this class by default only deals with 
    the specific environment it was designed for (FrozenLake8x8-v0). 
    However, the user can set Nstates and Nactions during the creation of an object, 
    which should allow this class to handle any Discrete action and state Space from the Gym (not tested).

    It is also possible to write a more generic code using Spaces: env.observation_space and env.action_space,
    see: https://gym.openai.com/docs/#spaces.

    A more elegant way would be to introduce an abstract class of an agent ("baseAgent") (e.g., using abc).
    That would allow us to check if an object 'agent' is an instance of the class baseAgent. 
    Due to limited time I won't take this approach here.
    """

    def __init__(self, gamma=0.99, alpha0=0.1, alpha_tau = 10000, beta0=3, beta_tau = 1e9, random_beta = True, state=0, Qinit=1, Nstates = 64, Nactions = 4):
        """Initialize the agent."""
        # Initialize tabular Q function.
        # Shape should be (# of states, # of actions) = (64, 4). 
        # Actions (from 0 to 3): left, down, right, up.
        self.Q  = np.zeros( (Nstates, Nactions) ) + Qinit # Start with an optimistic agent (to encourage exploration). 

        # Initialize learning parameters.
        self.gamma     = gamma
        self.alpha     = alpha0
        self.alpha0    = alpha0
        self.alpha_tau = alpha_tau
        
        # Initialize action selection parameters.
        self.beta0       = beta0
        self.beta_tau    = beta_tau
        self.random_beta = random_beta

        # Set variables that should be reset at the beginning of each episode.
        self.new_episode(episode = 0) 

    def new_episode(self, episode=None, state=0):
        # Initialize the state.
        self.s  = state
        # Initialize the action variable (None, because no action was taken so far)
        self.a  = None
        # Either add 1 to the episode number, or set it externally
        if episode is None:
            self.episode += 1
            episode = self.episode
        else:
            self.episode = episode
        # Set a new value of alpha.
        self.set_alpha(step = episode)
        # Set a new value of beta.
        self.set_beta(step = episode, random=self.random_beta)

    def select_action(self, greedy=False):
        """Select and return an action.

        Available policies:
        softmax and greedy. 

        Parameters:
        s -- current state
        Q -- tabular Q function (Q[state,action])
        beta -- inverse temperature of the softmax function
        greedy (default False) -- if True, a deterministic (greedy with respect to Q) selection is used (effectively beta=infinity) 
        
        Returns: an integer (action)
        """ 
        if greedy:
            self.a = np.argmax(self.Q[self.s,:]) 
            return self.a
        
        probs = np.exp(self.beta*self.Q[self.s,:])
        probs = probs/np.sum(probs)

        self.a = np.random.choice(len(probs), p=probs) 
        return self.a
    
    def update(self, obs, r, done, learn = True):
        """Update the state of the agent as well as the Q-function estimator (if learn==True).
        
        The agent must have taken an action before.
        """
        if learn:
            if self.a is None:
                raise Exception("Cannot update because the agent did not take any action so far. Call select_action() first.")
            # Update the Q function using the TD error.
            old_prediction = self.Q[self.s, self.a] 
            if done:
                # There will not be any next step here, so we should not 
                # bootstrap (add our prediction of future rewards).
                new_prediction = r 
            else:
                new_prediction = r + self.gamma*np.max(self.Q[obs,:])

            # Update the action-value function
            self.Q[self.s, self.a] += self.alpha*(new_prediction - old_prediction)

        # Update the state.
        self.s = obs

    def set_alpha(self, alpha=0, step = 0, scheduler=True):
        """Set the learning rate. 

        If scheduler is False alpha is set manually.
        Otherwise the argument step should be provided which 
        allows this function to determine the value of alpha according 
        to the scheduler.
        """ 
        if not scheduler:
            self.alpha = alpha
        else:
            # Currently only ~1/t scheduler implemented. 
            self.alpha = self.alpha0/(1 + step/self.alpha_tau)
        return self.alpha

    def set_beta(self, beta=0, step = 0, scheduler=True, random=False):
        """Set an inverse temperature for the softmax action selection. 

        If scheduler=False beta is set manually.
        Otherwise parameter step should be provided which allows 
        this function to determine the value of beta according 
        to the scheduler.
        Two schedulers are available:
        (1) non-random: beta increases linearly in time.
        (2) random: beta is generated randomly from an exponential distribution.
        """ 
        if not scheduler:
            self.beta = beta
        else:
            # Currently only ~1/t scheduler implemented.
            if random:
                self.beta = -self.beta0*np.log(np.random.rand())
            else: 
                self.beta = self.beta0 + step/self.beta_tau
        return self.beta

# ------------Handcrafted agent-----------------
class testAgent(discreteQlearningAgent):
    """A simple handcrafted agent solving the task defined in the OpenAI Gym FrozenLake8x8-v0 environment. 

    Strategy: 
        (1) go up if in the top row. This will lead to one of three outcomes: stay, left, or right.
        (2) go right if the the first column from the right. This will lead to one of three outcomes: stay, up, or down.
    Given this strategy, the agent should never move outside of of top-right strips. Thus, the agent will be "surprised" 
    to find itself in such a situation and will throw an exception.

    """

    def select_action(self, greedy=False):
        if self.s < 7:      # Top row excluding the top right corner
            self.a = 3      # go UP
        elif self.s%8 == 7: # The first column from the right 
            self.a = 2      # go RIGHT
        else:
            raise Exception(f"I did not expect this state: {self.s}")
        
        return self.a    

# ------------Trainer (gym wrapper) class--------------------
class gymTrainer:
    """A trainer class for OpenAI Gym. 

    Tested only in a single environment (FrozenLake8x8-v0), 
    but it should also work fine in other environments. 
    
    Methods:
    -------
    train: trains an agent. The agent can be specified during class instantation. 
    It can also  be provided in the train method; in this case the environment will store 
    that agent for future trainings and tests.
    
    test: tests an agent. A greedy policy of the agent will be used (agent's select_action method must provide a flag greedy).
    
    save_agent: save an agent to a file.

    load_agent: load an agent from a file or set the 'student' agent to an agent loaded externally.

    """
    def __init__(self, env, agent=None):
        """Initialize an instance of the gymTrainer class.

        env   -- gym environment, must be loaded externally.
        agent -- either an instance of discreteQlearningAgent class, 
        or a filename of a corresponding pickle. 
        """
        self.env   = env
        self.load_agent(agent)
        self.reset_logs()


    def train(self, agent=None, Nepisodes=1000, visualize=True, save_name=None):
        """Train an agent.

        agent can be:
        None -- use the agent that is stored in self.agent
        str  -- load the agent from a file 
        discreteQlearningAgent -- or, more generally, an instance of a class that provides methods new_episode, select_action, and update.

        Note that this method will replace self.agent with a new agent, if provided. 
        In this case it will also reset training logs.
        """
        if agent is not None:
            # Load a new agent.
            self.load_agent(agent)
            # Reset logs.
            self.reset_logs()
        else:
            if self.agent is None:
                raise Exception("No agent to train (self.agent is None)")

        agent = self.agent

        for i in range(Nepisodes):
            env.reset()
            try:
                agent.new_episode()
            except AttributeError:
                raise Exception("agent: method new_episode not implemented correctly.")
            # Alternate between greedy and stochastic policy. 
            # This should allow the agent to explore more, but also to 
            # make sure that it can obtain a positive reward more often. 
            # Additionally, this allows us to better visualize the learning progress. 
            # The observed performance of the greedy policy is a proxy to 
            # how well the Q-function has been learned so far.
            if i%2:
                greedy = True
            else:
                greedy = False
            for j in range(MAX_LEN_EPISODE):
                # Select an action.
                try:
                    action = agent.select_action(greedy=greedy)
                except AttributeError:
                    raise Exception("agent: method select_action not implemented correctly.")
                # Take a step in the environment using our selected action and collect observations.
                obs, r, done, info = env.step(action)
                # Update agent's state and Q-function.
                try:
                    agent.update(obs, r, done)
                except AttributeError:
                    raise Exception("agent: method update not implemented correctly.")
                # If the episode is done, update the history.
                if done:
                    if greedy:
                        self.log_greedy.append( (i, r, j+1) )
                    else:             
                        self.log_noisy.append( (i, r, j+1) )
                    break
                if j+1==MAX_LEN_EPISODE:
                    raise Exception(f"The length of the episode has hit the maximum number: {MAX_LEN_EPISODE}")
        if visualize:
            self.__visualize_training_progress(self.log_noisy, title="Stochastic policy")
            self.__visualize_training_progress(self.log_greedy,title="Greedy policy")
        
        if not save_name is None:
            # Save the agent in a (pickle) file.
            # We could also save the logs, but frankly I would rather 
            # use an external logger/tracking tool like Tensorboard or Neptune.
            self.save_agent(save_name)

        return self.log_noisy, self.log_greedy

    def test(self, agent=None, Nepisodes=1000, summary=True):
        """Train an agent.

        agent can be:
        None -- use the agent that is stored in self.agent
        str  -- load the agent from a file 
        an instance of a class that provides methods new_episode, select_action, and update, see discreteQlearningAgent class.

        In contrast to train method, this method will not change self.agent, so it can be used to test 
        different agents while keeping a single agent for training.
        """
        rhist = []
        lhist = []
        if agent is None:
            agent = self.agent
        else:
            agent = self.load_agent(agent, remember_agent=False)

        for i in range(Nepisodes):
            env.reset()
            
            try:
                agent.new_episode()
            except AttributeError:
                raise Exception("agent: method new_episode not implemented correctly.")

            for j in range(MAX_LEN_EPISODE):
                # Select an action according to the greedy policy.
                try:
                    action = agent.select_action(greedy=True)
                except AttributeError:
                    raise Exception("agent: method select_action not implemented correctly.")
                # Take a step in the environment using our selected action and collect observations.
                obs, r, done, info = env.step(action)
                # Update agent's state (but do not train!)
                try:
                    agent.update(obs, r, done, learn=False)
                except AttributeError:
                    raise Exception("agent: method update not implemented correctly.")

                if done:
                    rhist.append(r)
                    lhist.append(j+1)
                    break
                if j+1==MAX_LEN_EPISODE:
                    raise Exception(f"The length of the episode has hit the maximum number: {MAX_LEN_EPISODE}")
        
        if summary:
            rhist_np = np.asarray( rhist )
            r_mean = np.mean( rhist_np )
            #r_std  = np.std( rhist_np ) # We can infer this from r_mean for binary (Bernoulli) random variable
            
            lhist_np = np.asarray( lhist )
            l_mean = np.mean( lhist_np )
            l_std  = np.std( lhist_np )
        
            print(f"Win rate: {100*r_mean:.1f} \u00B1 {100*3*np.sqrt(r_mean*(1-r_mean))/np.sqrt(len(rhist_np)):.1f} %")
            print(f"Average number of steps: {l_mean:.1f} \u00B1 {3*l_std/np.sqrt(len(lhist_np)):.1f}")
        return rhist, lhist

    def save_agent(self, fname):
        """Save an object (agent) to a file (fname).
        
        Note that this function can save other objects too, 
        since we are not checking here if the objects is an 
        instance of a class that is consistent with 
        the agent class. 
        """
        pickle.dump(self.agent, file=open(fname, "wb"))
        
    def load_agent(self, agent_name, remember_agent=True):
        """Load and return an object (agent). 

        If agent_name is a string, load the agent from a pickle file.
        If loading the file fails, set agent as None.
 
        If agent_name is not a string this function assumes 
        that it is an instance of the class discreteQlearningAgent 
        and loads it as such (setting self.agent to agent_name).

        The flag remember_agent is used to determine if self.agent should be replaced. 
        This is useful to switch between two alternative behaviors: 
        loading an agent temporarily (for tests), or loading a new agent for training. 

        Note that this function can load other objects too, 
        since we are not checking here if the objects is an 
        instance of a class that is consistent with 
        the agent class. 
        
        TO DO: check if the loaded object is an instance of the class...
        """

        if isinstance(agent_name, str):
            try:
                agent = pickle.load(file=open(agent_name, "rb"))
            except FileNotFoundError:
                print(f"File {agent_name} does not exist.") 
                agent = None
        else:
            agent = agent_name
        
        if remember_agent:
            self.agent = agent

        return agent
    
    def reset_logs(self):
        """Empty the lists containing logs."""
        self.log_greedy = []
        self.log_noisy  = []

    def __visualize_training_progress(self, log_list, title="", tau1=100, tau2=1000):
        """ Visualize training progress. """
        
        # List of tuples --> ndarray.
        # log[:,0] -- 'time steps' (episode #)
        # log[:,1] -- rewards
        # log[:,2] -- length of the episode
        log = np.array( log_list )

        fig, ax = plt.subplots(1,2, figsize=(10,4))
        # Plot (low-pass filtered) rewards
        ax[0].plot(log[:,0], self.__low_pass(log[:,1], tau=tau1))
        ax[0].plot(log[:,0], self.__low_pass(log[:,1], tau=tau2))
        ax[0].set_xlabel('# of episodes')
        ax[0].set_ylabel('average (low-pass filtered) reward')
        # Plot (low-pass filtered) number of steps 
        ax[1].plot(log[:,0], self.__low_pass(log[:,2], tau=tau1))
        ax[1].plot(log[:,0], self.__low_pass(log[:,2], tau=tau2))
        ax[1].set_xlabel('# of episodes')
        ax[1].set_ylabel('average # of steps per episode')

        fig.suptitle(title, fontsize=14)

    def __low_pass(self, x, tau=1):
        """Calculate and return a first-order linear low-pass filter of a 1D time series. 

        Parameters:
        x -- 1D numpy array with the time series to be filtered
        tau -- time constant of the filter
        
        Returns: 1D numpy array with the filtered signal (same length as x)
        """
        y = np.empty( (len(x)) )
        y[0] = np.mean(x[:tau])     # Initialize the first element with an average.
        a = 1/tau

        for i in range(len(x)-1):
            y[i+1] = (1-a)*y[i] + a*x[i]

        return y

# Create an instance of the FrozeLake8x8-v0 environment
env = gym.make('FrozenLake8x8-v0')#, map_name=None)
# Instantiate an agent (discreteQlearningAgent)
agent = discreteQlearningAgent(alpha_tau=3000)
# Instantiate a gymTrainer
trainer = gymTrainer(env, agent)
# It is also possible to specify the path with a pickle containing the agent:
# trainer = gymTrainer(env, "agentQ.pickle")

# Train the agent for Nepisodes steps. By default this will also visualize the training.
# If save_name is specified as a string (filename), 
# the agent will be automatically saved to the file (pickle) at the end of the training. 
trainer.train(Nepisodes=20000, save_name="agent0.pickle")

# Test the trained agent in the trainer.
print("== Trained agent (tabular Q-learning): ==")
trainer.test()

# Test another agent (loaded externally) using the same trainer. Note that this will not update the agent for future training.
print("== Simple handcrafted agent: ==")
agentHC = testAgent()   # Handcrafted agent
trainer.test(agentHC)

# Test another agent loaded directly from a specified path.
trainer.test("agent0.pickle")

# Save the trained agent whenever needed. 
trainer.save_agent("agent1.pickle");

In [ ]:
trainer_new = gymTrainer(env, "agentQ.pickle")
trainer_new.test();

In [ ]:
trainer.test(6)

In [ ]:
print(np.__version__)
import matplotlib as mpl
print(mpl.__version__)
print(gym.__version__)
print(pickle.format_version)